# Perform unconstrained runs

In [1]:
import sys
import yaml
import numpy as np
import pandas as pd
from netCDF4 import Dataset
from scipy.interpolate import interp1d
from tqdm.notebook import tqdm

In [2]:
# mce is available in https://github.com/tsutsui1872/mce 
sys.path.append('../mce')

In [3]:
from mce.core.forcing import RfAll

In [4]:
from src.util import RetrieveGitHub, df2nc
from src.tlm import DriverMCE

In [5]:
owner = 'IPCC-WG1'
repo = 'Chapter-7'
repo_ch7 = RetrieveGitHub(owner, repo, './datain')

## Forcing input with uncertainties

In [6]:
path = repo_ch7.retrieve('data_output/AR6_ERF_1750-2019.csv')
df_erf_hist = pd.read_csv(path).set_index('year')

[2024-07-07 09:55:13 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/AR6_ERF_1750-2019.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/AR6_ERF_1750-2019.csv on 2024-06-13


In [7]:
path = repo_ch7.retrieve('data_output/emissions_ceds_update_plus_bb.csv')
df_emis = pd.read_csv(path, index_col=0)

[2024-07-07 09:55:16 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/emissions_ceds_update_plus_bb.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/emissions_ceds_update_plus_bb.csv on 2024-06-13


In [8]:
nsize = 50000

ncf = Dataset('./dataout/fair_samples_forcing.nc')

df_erf_scale = pd.DataFrame({
    k: ncf.variables[f'scale_normals__{k}'][:nsize]
    for k in [
        'co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
        'contrails', 'bc_on_snow', 'land_use', 'volcanic', 'solar',
    ]
})

trend_solar = ncf.variables['trend_solar'][:nsize].filled()

aer_coeff = {
    k: ncf.variables[k][:nsize].filled()
    for k in ['beta_so2', 'beta_bc', 'beta_oc', 'beta_nh3', 'beta']
}
aer_coeff['aci_coeffs'] = np.array([
    ncf.variables['aci_coeffs__0'][:nsize].filled(),
    ncf.variables['aci_coeffs__1'][:nsize].filled(),
]).T

ncf.close()

## TLM parameter ensemble

In [9]:
ncf = Dataset('./dataout/parms_sample.nc')

method_order = [x for x in ncf.variables['Method'][:]]
names = ['q4x', 'q2x', 'lamg', 'cmix', 'cdeep', 'gamma_2l', 'eff', 'co2_beta']

df = [ncf.variables[k][:].filled(np.nan) for k in names]
df_parms_sample = pd.DataFrame(
    np.hstack(df),
    columns=pd.MultiIndex.from_product([names, method_order]),
)

ncf.close()

In [10]:
len(df_parms_sample) == nsize

True

## CO2 forcing scales

In [11]:
obj_rf = RfAll()

# Reference values of CO2 in 2019 and 4xCO2 forcing
cco2_pi = obj_rf.parms_ar6_ghg.C0_1750
obj_rf.parms.update(ccref=cco2_pi)

cco2_2019 = 409.85
cn2o_2019 = 332.091
erf_co2_2019 = obj_rf.c2erf_ar6('CO2', cco2_2019, cn2o=cn2o_2019)
q4x_ref = obj_rf.c2erf_ar6('CO2', cco2_pi*4.)
cco2_pi, cco2_2019, erf_co2_2019, cco2_pi*4., q4x_ref

(278.3, 409.85, 2.156277925173476, 1113.2, 8.259783657536742)

In [12]:
df_scale_co2 = pd.concat([
    df_parms_sample.loc[:, ('q4x', method)].div(q4x_ref).rename(method)
    for method in ['ar6', 's21']
], axis=1)

In [13]:
df_scale_co2.apply(['mean', 'std'])

,ar6,s21
mean,0.943026,0.953481
std,0.128072,0.134166


For MCE-2l, use equivalent CO2 concentrations, converted from well-mixed GHG ERF
with the AR6 formula of CO2 ERF.
Here, define an interpolation object to get the equivalent CO2 concentrations.

In [14]:
# Nominal range of well-mixed GHG ERF across the SSP scenarios
xlim = (-1., 12.)

# y-value: equivalent CO2 concentrations
alpha = erf_co2_2019 / np.log(cco2_2019/cco2_pi)
yp = cco2_pi * np.exp(np.linspace(*xlim, **{'num': 500}) / alpha)

# x-value: well-mixed GHG ERF
xp = obj_rf.c2erf_ar6('CO2', yp)

interp_cco2 = interp1d(xp, yp)

## Ensemble member loop

In [15]:
driver_mce = DriverMCE()

In [16]:
cats = [
    'co2', 'ch4', 'n2o', 'other_wmghg', 'o3',
    'h2o_stratospheric', 'contrails', 'bc_on_snow', 'land_use',
    'volcanic', 'solar',
]
cats_aerosol = [
    'aerosol-radiation_interactions',
    'aerosol-cloud_interactions',
]
cats_nonco2 = cats[1:] + cats_aerosol

In [17]:
df_emis_a = df_emis.sub(df_emis.loc[1750])

In [18]:
with open('./src/variables.yml') as f1:
    var_atts = yaml.safe_load(f1)

In [19]:
variable_order = ['tg', 'ohc']

df = pd.DataFrame(
    0.,
    index=pd.Index([0], name='Member'),
    columns=pd.MultiIndex.from_product([
        variable_order,
        method_order,
        df_erf_hist.index,
    ], names=['Variable', 'Method', 'Year'])
)
path_out = './dataout/unconstrained_run.nc'
df2nc(path_out, df, var_atts)

ncf = Dataset(path_out, 'r+')

[2024-07-07 09:56:13 src.util] INFO:dataout/unconstrained_run.nc is created


In [ ]:
for im in tqdm(range(nsize)):
    dfin = df_erf_hist.loc[:, cats].mul(df_erf_scale.loc[im, cats])

    dfin['solar'] += np.linspace(0, trend_solar[im], dfin.shape[0])

    dfin['aerosol-radiation_interactions'] = (
        df_emis_a['SO2'] * aer_coeff['beta_so2'][im] * 32./64.
        + df_emis_a['BC'] * aer_coeff['beta_bc'][im]
        + df_emis_a['OC'] * aer_coeff['beta_oc'][im]
        + df_emis_a['NH3'] * aer_coeff['beta_nh3'][im]
    )

    d1 = -aer_coeff['beta'][im] * np.log(
        1.
        + df_emis['SO2'].mul(32./64.) / aer_coeff['aci_coeffs'][im, 0]
        + df_emis[['BC', 'OC']].sum(axis=1) / aer_coeff['aci_coeffs'][im, 1]
    )
    dfin['aerosol-cloud_interactions'] = d1.sub(d1.loc[1750])

    df_parms = df_parms_sample.loc[im].unstack(0)

    ncf.variables['Member'][im] = im

    for jm, method in enumerate(method_order):
        p1 = df_parms.loc[method].dropna()
        driver_mce.calib(p1)

        if method == 'ar6_orig':
            din_co2 = dfin['co2']
        elif method in ['ar6', 's21']:
            # Perturbed CO2 forcing is further scaled
            din_co2 = dfin['co2'] * df_scale_co2.loc[im, method]
        elif method == 'mce-2l':
            # Convert CO2 forcing to equivalent CO2 concentrations
            # and calculate forcing with the MCE CO2 scheme
            obj_rf.parms.update(
                alpha=p1['q4x'] / (np.log(4.) * p1['co2_beta']),
                beta=p1['co2_beta'],
            )
            din_co2 = obj_rf.c2erf(interp_cco2(dfin['co2'].values))
        else:
            raise ValueError(f'unexpected method {method}')

        din = din_co2 + dfin[cats_nonco2].sum(axis=1)
        dfout = driver_mce.run(din)

        for vn, d1 in dfout.items():
            if vn not in variable_order:
                continue
            ncf.variables[vn][im, jm, :] = d1.values

ncf.close()